In [1]:
from generate_training_data import functions, questions_dict
from dotenv import load_dotenv

import os

load_dotenv()

True

In [2]:
functions[0]

{'name': 'adjust_temperature',
 'description': 'Adjust the temperature in a specified zone of the car.',
 'parameters': {'type': 'object',
  'properties': {'zone': {'type': 'string',
    'enum': ['front', 'rear', 'all'],
    'description': 'The zone where the temperature will be adjusted.',
    'default': 'all'},
   'temperature': {'type': 'number',
    'description': 'The target temperature for the specified zone in degrees Celsius.'}},
  'required': ['temperature'],
  'optional': ['zone']}}

In [3]:
import numpy as np
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import AzureChatOpenAI


/anaconda/envs/ml-gpu/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
class ResultSchema(BaseModel):
    # function_called_with_params: str = Field(..., description="Exact function with parameters to be called")
    user_query: str = Field(..., description="User query that caused the function to be called")
    
    def model_json_schema():
        return {
            "type": "object",
            "properties": {
                # "function_called_with_params": {"type": "string"},
                "user_query": {"type": "string"},
            },
            "required": ["function_called", "user_query"],
        }



In [9]:
class GPTQueryGenerator:
    def __init__(self):
        self.llm = AzureChatOpenAI(
                azure_endpoint = os.getenv("AZURE_ENDPOINT"),
                openai_api_key = os.getenv("OPENAI_API_KEY"),
                deployment_name=os.getenv("DEPLOYMENT_NAME"),  # or your deployment
                openai_api_version=os.getenv("OPENAI_API_VERSION"),  # or your api version
                temperature=0.9,
                max_tokens=None,
                timeout=None,
                max_retries=2,
            )

        self.parser = PydanticOutputParser(pydantic_object=ResultSchema)
        
        self.prompt = PromptTemplate(template = '''
                                     You have been provided with a complete function description.A function has multiple parameters and each 
                                     parameter type and possible values for that parameter, have been provided too, if there is a limit in choice. \nThere are some 
                                     required and some optional parameters.
                                     \nYou have to come up with a user command that will call this function.\nAssume you are someone driving the car when generating the command.\n 
                                     The user command need not have optional parameters in it.\n
                                     Function Description: {function_description}\n
                                     \n{format_instructions}\n
                                      
                                     ''', 
                                     input_variables=["function_description"],
                                     partial_variables={"format_instructions": self.parser.get_format_instructions()}
                                     )
    
    def get_result(self, function_description: str) -> ResultSchema:
        chain = self.prompt | self.llm | self.parser
        response = chain.invoke({"function_description": function_description})
        print(response)
        # try:
        #     response = chain.invoke({"function_description": function_description})
        #     print(response)
        
        # except Exception as e:
        #     return ResultSchema(function_called="None", user_query="None")
        
        return response

In [10]:
query_generator = GPTQueryGenerator()
for i in range(10):
    try:
        query_generator.get_result(functions[0])
    except Exception as e:
        pass

user_query='Set the temperature to 22 degrees.'
user_query='Set the temperature to 22 degrees Celsius in the front zone.'
user_query='Set the temperature to 22 degrees.'
user_query='Adjust the temperature to 22 degrees Celsius.'
user_query='Set the temperature to 22 degrees in the front zone.'
user_query='Set the temperature to 22 degrees Celsius.'
user_query='Adjust the temperature to 22 degrees Celsius in the front zone.'
user_query='Set the temperature to 22 degrees in the front zone.'
user_query='Set the temperature to 22 degrees Celsius'


In [ ]:
functions[0]